In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
import pandas as pd
import numpy as np
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_102"; Java(TM) SE Runtime Environment (build 1.8.0_102-b14); Java HotSpot(TM) 64-Bit Server VM (build 25.102-b14, mixed mode)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/n7/20vqb2_91k9f4br462p_wg2c0000gn/T/tmpqgj3mp_n
  JVM stdout: /var/folders/n7/20vqb2_91k9f4br462p_wg2c0000gn/T/tmpqgj3mp_n/h2o_maverick_started_from_python.out
  JVM stderr: /var/folders/n7/20vqb2_91k9f4br462p_wg2c0000gn/T/tmpqgj3mp_n/h2o_maverick_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.16.0.3
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_maverick_12na33
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import math
import seaborn as sns
sns.set()


from copy import deepcopy
from collections import defaultdict

# Columns with categorical data that needs to be encoded
categorical_cols = ['serve', 'hitpoint', 'outside.sideline', 
                    'outside.baseline', 'same.side', 'server.is.impact.player', 
                    'gender' , 'previous.hitpoint']

# Columns in the Data That Should Be Scaled
scaled_data = ['rally', 'speed', 'net.clearance', 'distance.from.sideline', 'depth', 'player.distance.travelled', 
               'player.impact.depth', 'player.impact.distance.from.center', 'player.depth', 
               'player.distance.from.center', 'previous.speed', 'previous.net.clearance', 
               'previous.distance.from.sideline', 'previous.depth', 'opponent.depth', 
               'opponent.distance.from.center', 'previous.time.to.net', 'player.impact.distance.from.center']

# Columns to be dropped
dropcols = ['id', 'train', 'gender', 'same.side', 'server.is.impact.player', 'outside.baseline', 
            'previous.hitpoint', 'hitpoint', 'rally', 'serve', 'outside.sideline', 'player.distance.from.center',
            'player.distance.from.each.other.length', 'player.distance.from.each.other.width',
           'player.impact.distance.from.center',
           'opponent.distance.from.center',  'bw.player.distance.penultimate'
           ,'net.clearance.difference', 'depth']

# Load Data
raw_mens_train = pd.read_csv('tennis_data/mens_train_file.csv')
raw_mens_test = pd.read_csv("tennis_data/mens_test_file.csv")
raw_womens_train = pd.read_csv('tennis_data/womens_train_file.csv')
raw_womens_test = pd.read_csv("tennis_data/womens_test_file.csv")

# NO IMPACTS- gender, train, id rally
# LOW IMPACT- 'server.is.impact.player', 'previous.hitpoint', 'same.side',  'outside.baseline',
# MED IMPACT- 'previous.time.to.net', 'opponent.distance.from.center', 'opponent.depth',  'previous.depth',  
# 'previous.distance.from.sideline', 'previous.net.clearance', 'player.distance.from.center',  'player.depth',
# 'player.impact.distance.from.center', 'player.distance.travelled', 'distance.from.sideline',
# HIGH IMPACT- 'previous.speed', 'player.impact.depth', 'depth', 'speed'


# Feature Engineering

def feature_engineer(data):
    data['speed.difference'] = data['speed'] - data['previous.speed']
    data['net.clearance.difference'] = data['net.clearance'] - data['previous.net.clearance']
    data['true.distance.from.sideline'] = [dist if not boolean else (dist * -1) for dist, boolean in 
                                                 zip(data['distance.from.sideline'], 
                                                     data['outside.sideline'])]

    data['true.distance.from.baseline'] = [dist if not boolean else (dist * -1) for dist, boolean in 
                                                    zip(data['depth'], 
                                                    data['outside.baseline'])]

    # Opponent distance to net + player distance to net
    data['player.distance.from.each.other.length'] = data['player.depth'] + data['opponent.depth']

    # Opponent distance from center + player distance to center
    data['player.distance.from.each.other.width'] = [math.fabs(p_center - o_center)
                                                             if boolean 
                                                             else p_center + o_center
                                                             for p_center, o_center, boolean in 
                                                             zip(data['player.distance.from.center'], 
                                                                 data['opponent.distance.from.center'], 
                                                                 data['same.side'])]

    # Straight line distance between players
    data['bw.player.distance.penultimate'] = [math.hypot(length, width)
                                                 for length, width in 
                                                 zip(data['player.distance.from.each.other.length'], 
                                                     data['player.distance.from.each.other.width'])]

    # Distance penultimate shot was made from net- previous to net (s) * penultimate shot (m/s)
    data['player.from.net.penultimate'] = data['previous.speed'] * data['previous.time.to.net']
    return data
    
mens_train = feature_engineer(raw_mens_train)
mens_test = feature_engineer(raw_mens_test)

womens_train = feature_engineer(raw_womens_train)
womens_test = feature_engineer(raw_womens_test)

# Encode Categorical Data
def encode(train, test):
    # Retain All LabelEncoder as a dictionary
    d = defaultdict(LabelEncoder)
    # Encode all the columns
    train[categorical_cols] = train[categorical_cols].apply(lambda x: d[x.name].fit_transform(x))
#     print(train)
    test_ids = test['id']
    # Inverse the encoding
    # data.apply(lambda x: d[x.name].inverse_transform(x))
    
    # Using dictionary d to label future data
    temp = deepcopy(categorical_cols)
    
    e = deepcopy(d)
    
    
    test[temp] = test[temp].apply(lambda x: e[x.name].transform(x))
    train = train.drop(dropcols, axis=1)
    test = test.drop(dropcols + ['outcome'], axis=1)
    return train, test, test_ids, d
    

mens_train, mens_test, mens_test_ids , mens_d = encode(mens_train, mens_test)
womens_train, womens_test, womens_test_ids , womens_d = encode(womens_train, womens_test)

In [3]:
mens_train_hf = h2o.H2OFrame(mens_train)
mens_test_hf = h2o.H2OFrame(mens_test)
womens_train_hf = h2o.H2OFrame(womens_train)
womens_test_hf = h2o.H2OFrame(womens_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
mens_train_hf["outcome"] = mens_train_hf["outcome"].asfactor()
mens_gb_model = H2OGradientBoostingEstimator(nfolds=5, keep_cross_validation_predictions=True)
mens_gb_model.train(x=mens_train_hf.names[0:-5] + mens_train_hf.names[-4:], y=mens_train_hf.names[-5], 
            training_frame=mens_train_hf)
mens_pred = mens_gb_model.predict(mens_test_hf)
mens_test_pred = mens_pred[['FE', 'UE', 'W']]
mens_gb_model

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1515782635053_2


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.050096392424037045
RMSE: 0.2238222339805343
LogLoss: 0.19064211480179957
Mean Per-Class Error: 0.059159484597654466
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



FE,UE,W,Error,Rate
1065.0,112.0,21.0,0.1110184,"133 / 1,198"
49.0,2020.0,49.0,0.0462701,"98 / 2,118"
9.0,25.0,1650.0,0.0201900,"34 / 1,684"
1123.0,2157.0,1720.0,0.053,"265 / 5,000"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.947
2,0.9952000
3,1.0



ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.09665829892996433
RMSE: 0.3108991780786246
LogLoss: 0.32495742074915446
Mean Per-Class Error: 0.1421176169832041
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.8717894,0.0057429,0.8689176,0.8657588,0.8616558,0.8821179,0.8804972
err,0.1282106,0.0057429,0.1310824,0.1342412,0.1383442,0.1178821,0.1195029
err_count,128.0,4.753946,132.0,138.0,127.0,118.0,125.0
logloss,0.3254935,0.0130206,0.3122203,0.3394042,0.3548164,0.3075940,0.3134327
max_per_class_error,0.2665095,0.0156390,0.2727273,0.2642276,0.3008850,0.2317597,0.2629482
mean_per_class_accuracy,0.8577616,0.0060713,0.8549025,0.8530713,0.8457896,0.8690335,0.8660111
mean_per_class_error,0.1422384,0.0060713,0.1450975,0.1469286,0.1542104,0.1309665,0.1339889
mse,0.0968017,0.0037313,0.0939444,0.1014838,0.1046186,0.0911,0.0928615
r2,0.8292676,0.0064314,0.8321122,0.8190639,0.8182142,0.8406790,0.8362688
rmse,0.3110153,0.0059651,0.3065035,0.3185652,0.3234480,0.3018278,0.3047319


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2018-01-12 13:45:20,5.458 sec,0.0,0.6666667,1.0986123,0.649
,2018-01-12 13:45:20,5.484 sec,1.0,0.6270294,0.9871163,0.1402
,2018-01-12 13:45:20,5.513 sec,2.0,0.5884949,0.8901064,0.13
,2018-01-12 13:45:20,5.536 sec,3.0,0.5533856,0.8093553,0.1336
,2018-01-12 13:45:20,5.557 sec,4.0,0.5241815,0.7463691,0.1268
---,---,---,---,---,---,---
,2018-01-12 13:45:21,6.222 sec,46.0,0.2315372,0.2020510,0.0576
,2018-01-12 13:45:21,6.237 sec,47.0,0.2296747,0.1992664,0.0564
,2018-01-12 13:45:21,6.251 sec,48.0,0.2277441,0.1964266,0.0546
,2018-01-12 13:45:21,6.265 sec,49.0,0.2257387,0.1935532,0.0538



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
net.clearance,3881.5683594,1.0,0.3038216
true.distance.from.sideline,2581.1420898,0.6649740,0.2020335
previous.time.to.net,1172.9927979,0.3021956,0.0918135
true.distance.from.baseline,993.3050537,0.2559030,0.0777489
previous.distance.from.sideline,682.8216553,0.1759139,0.0534464
speed.difference,673.8191528,0.1735946,0.0527418
player.impact.depth,626.4588623,0.1613932,0.0490347
player.from.net.penultimate,404.4360962,0.1041940,0.0316564
player.distance.travelled,395.5386658,0.1019018,0.0309600
speed,341.0171204,0.0878555,0.0266924


In [14]:
from sklearn.model_selection import GridSearchCV
# Paramter Tuning
# n_estimators = [100, 200, 300, 400, 500]
# learning_rate = [0.0001, 0.001, 0.01, 0.1]

# def parameter_tuning():

cv_params = {'ntrees': [0.1, 0.3, 0.8, 1]}
ind_params = {'seed':0, 'learn_rate': 0.1, 'max_depth': 5}
optimized_GBM = GridSearchCV(H2OGradientBoostingEstimator(**ind_params), 
                             cv_params, scoring = 'neg_log_loss', cv = 5, n_jobs = -1, verbose=3) 

optimized_GBM.fit(mens_train_X., mens_train_y)
#     grid_search = GridSearchCV(xgb_model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
#     grid_result = grid_search.fit(mens_train_X, mens_train_y)
#     print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


NameError: name 'mens_train_X' is not defined

In [11]:
womens_train_hf["outcome"] = womens_train_hf["outcome"].asfactor()
womens_gb_model = H2OGradientBoostingEstimator(nfolds=5)
womens_gb_model.train(x=womens_train_hf.names[0:-5] + womens_train_hf.names[-4:], y=womens_train_hf.names[-5], 
            training_frame=womens_train_hf)
pred = womens_gb_model.predict(womens_test_hf)
womens_test_pred = pred[['FE', 'UE', 'W']]
womens_gb_model

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1515782635053_3


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.05666233096184892
RMSE: 0.23803850730889933
LogLoss: 0.21093726072289565
Mean Per-Class Error: 0.07132103620832843
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



FE,UE,W,Error,Rate
930.0,116.0,28.0,0.1340782,"144 / 1,074"
59.0,2125.0,74.0,0.0589017,"133 / 2,258"
9.0,26.0,1633.0,0.0209832,"35 / 1,668"
998.0,2267.0,1735.0,0.0624,"312 / 5,000"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9376
2,0.9934000
3,1.0



ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.10210054227616733
RMSE: 0.3195317547226994
LogLoss: 0.3452985435833766
Mean Per-Class Error: 0.1476169594531064
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.8722352,0.0030489,0.8760081,0.8729228,0.875,0.8733401,0.8639053
err,0.1277647,0.0030489,0.1239919,0.1270772,0.125,0.1266599,0.1360947
err_count,127.8,4.0099874,123.0,130.0,124.0,124.0,138.0
logloss,0.3452800,0.0092369,0.3229123,0.3410263,0.3599724,0.3463561,0.3561327
max_per_class_error,0.2825186,0.0220781,0.2441315,0.2476191,0.3063063,0.2932692,0.3212670
mean_per_class_accuracy,0.8527913,0.0045116,0.8621159,0.8563592,0.8506101,0.852004,0.8428673
mean_per_class_error,0.1472087,0.0045116,0.1378841,0.1436408,0.1493898,0.1479960,0.1571327
mse,0.1020846,0.0024634,0.0954435,0.1025557,0.1042485,0.1027367,0.1054383
r2,0.8086996,0.0055080,0.8176063,0.8065341,0.8150565,0.8089754,0.7953253
rmse,0.3194593,0.0038947,0.3089393,0.3202432,0.3228754,0.3205257,0.3247127


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2018-01-12 13:46:37,5.058 sec,0.0,0.6666667,1.0986123,0.6306
,2018-01-12 13:46:37,5.073 sec,1.0,0.6210775,0.9714702,0.1342
,2018-01-12 13:46:37,5.087 sec,2.0,0.5801448,0.8706876,0.1338
,2018-01-12 13:46:37,5.103 sec,3.0,0.5436423,0.7888026,0.1312
,2018-01-12 13:46:37,5.122 sec,4.0,0.5113177,0.7210450,0.1268
---,---,---,---,---,---,---
,2018-01-12 13:46:37,5.795 sec,46.0,0.2427770,0.2186890,0.0652
,2018-01-12 13:46:37,5.807 sec,47.0,0.2416010,0.2168944,0.0634
,2018-01-12 13:46:37,5.820 sec,48.0,0.2402410,0.2145576,0.063
,2018-01-12 13:46:37,5.836 sec,49.0,0.2394215,0.2132132,0.0626



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
net.clearance,3589.0742188,1.0,0.2895878
true.distance.from.sideline,2606.7263184,0.7262949,0.2103261
speed.difference,1451.3659668,0.4043845,0.1171048
true.distance.from.baseline,1277.8343506,0.3560345,0.1031033
previous.distance.from.sideline,635.2180786,0.1769866,0.0512532
previous.time.to.net,530.6887817,0.1478623,0.0428191
speed,449.8169250,0.1253295,0.0362939
player.impact.depth,305.6230774,0.0851537,0.0246595
player.from.net.penultimate,298.3027039,0.0831141,0.0240688
player.distance.travelled,271.4027710,0.0756192,0.0218984


In [12]:
def append_gender(data, gender):
    return str(data) + '_' + str(gender)

def create_column_ids(mens_id, womens_id):
    mens_test_id_col = mens_id.apply(append_gender, args=('mens',))
    womens_test_id_col = womens_id.apply(append_gender, args=('womens',))

    combined_id = np.concatenate((mens_test_id_col, womens_test_id_col))
    
    return pd.DataFrame(combined_id)

column_ids = create_column_ids(mens_test_ids, womens_test_ids)

combined_test_predictions = pd.concat([mens_test_pred.as_data_frame(), womens_test_pred.as_data_frame()], axis=0)
combined_test_predictions.columns = ['FE', 'UE', 'W']
combined_test_predictions.reset_index(inplace=True, drop=True)
combined_test_predictions.head()

,FE,UE,W
0,0.046278,0.946677,0.007045
1,0.018474,0.919762,0.061765
2,0.029493,0.962135,0.008372
3,0.039425,0.950657,0.009918
4,0.217374,0.769716,0.012910


In [13]:
def format_submission(predictions):
    format_file = pd.read_csv('tennis_data/AUS_SubmissionFormat.csv')
#     format_file['submission_id']
    # result = create_column_ids(mens_test_id, womens_test_id, result)
    final = pd.concat([column_ids, format_file[['train']], predictions], axis=1, )
    final.columns = ['submission_id', 'train', 'FE', 'UE', 'W']
    final = final.set_index(list(final[['submission_id']])).T
    correct_order = list(format_file['submission_id'])
    final_sorted = final[correct_order].T.reset_index()
    cols = ['submission_id', 'train', 'UE', 'FE', 'W']
    final_sorted = final_sorted[cols]
    return final_sorted

final_submission = format_submission(combined_test_predictions)
final_submission.head()
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
def save_file(data):
    data.to_csv('Submissions/SubmissionGB' + timestr + '.csv', index=False)
save_file(final_submission)